# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv', sep=',')

display(df.head())
print()                          # разделим пустой строчкой
df.info()                        # выводим на экран информацию о файле
print()                          # разделим пустой строчкой
print(df.columns)                # посмотрим правильность наименования столбцов

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')


Прочитав файл видим, что в нем 5 столбцов:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)
и 3214 строк.

Название столбцов правильное. Тип данных в столбцах правильный.

Пустых значений нет.

## Разбейте данные на выборки

Тестовой выборки в нашем случае нет, поэтому будем разбивать на три части: обучающую, валидационную и тестовую. 
Размеры тестового и валидационного наборов сделаем одинаковые. Исходные данные разобъем в соотношении 3:1:1.

In [3]:
train_df, test_df = train_test_split(df, test_size = 0.4)          # выделили под test_df 0,4 df, под train_df идет 0,6 df

In [4]:
test_df, valid_df = train_test_split(test_df, test_size = 0.5)     # выделили на test_df и valid_df по 0,2 df 

По каждой выборке извлечем признак и целевой признак

In [5]:
train_features = train_df.drop(['is_ultra'], axis=1) 
train_target = train_df['is_ultra'] 

In [6]:
test_features = test_df.drop(['is_ultra'], axis=1) 
test_target = test_df['is_ultra'] 

In [7]:
valid_features = valid_df.drop(['is_ultra'], axis=1)  
valid_target = valid_df['is_ultra'] 

Разбили данные на три выборки:
- тренировочную (train),
- тестовую (test),
- валидную (valid).

In [8]:
print('Размер тренировочной выборки:', train_df.shape[0])
print('Размер тестовой выборки :', test_df.shape[0])
print('Размер валидной выборки :', valid_df.shape[0])

Размер тренировочной выборки: 1928
Размер тестовой выборки : 643
Размер валидной выборки : 643


Получили размеры выборок. Их соотношение, как мы и планировали: 3:1:1.

## Исследуйте модели

### Первой моделью рассмотрим дерево решений

In [9]:
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(train_features, train_target)
    accuracy = model.score(valid_features, valid_target)
    if accuracy > best_accuracy:
        best_accuracy_dec_tree = accuracy
        best_depth = depth
print("Доля правильных ответов:", best_accuracy_dec_tree, "    Глубина дерева:", best_depth, )

Доля правильных ответов: 0.7480559875583204     Глубина дерева: 14


### Следующей рассмотрим случайный лес

In [10]:
best_accuracy = 0
best_depth = 0
for depth in range(1,15):                                               # максимальная глубина деревьев
    for est in range(5,50,5):                                           # число «деревьев» в «случайном лесу»
        for sample in range(2,5):                                       # минимальное число объектов в листьях
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=412)
            model.fit(train_features, train_target)
            accuracy = model.score(valid_features, valid_target)
            if accuracy > best_accuracy:
                best_accuracy_ranfor = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Доля правильных ответов:", best_accuracy_ranfor, " Глубина дерева:", best_depth,
      " Количество деревьев:", best_est, " Минимальное число объектов в листье:", best_sample, )

Доля правильных ответов: 0.80248833592535  Глубина дерева: 14  Количество деревьев: 45  Минимальное число объектов в листье: 4


### Логистическая регрессия

In [11]:
model = LogisticRegression(random_state=12345, solver = 'liblinear')  # инициализация модели логистической регрессии
model.fit(train_features, train_target)                               # обучение модели на тренировочной выборке
accuracy_logreg = model.score(valid_features, valid_target)           # получение метрики качества модели на валидационной выборке

print("Доля правильных ответов:", accuracy_logreg)

Доля правильных ответов: 0.7340590979782271


Рассмотрели три модели.
Выяснили, что наилучший показатель с 0.79 у модели "Случайный лес" с количеством деревьев 45, глубиной 14 и минимальным числом объектов в листье 4.

## Проверьте модель на тестовой выборке

### Проверим модель на дереве решений

In [12]:
model = DecisionTreeClassifier(random_state=12345, max_depth=5)
model.fit(train_features, train_target)
accuracy = model.score(test_features, test_target)
print("Доля правильных ответов на тестовой выборке:", accuracy)
print("Доля правильных ответов на валидной выборке:", best_accuracy_dec_tree)

Доля правильных ответов на тестовой выборке: 0.7791601866251944
Доля правильных ответов на валидной выборке: 0.7480559875583204


### Проверим модель на случайном лесе

In [13]:
model = RandomForestClassifier(max_depth=7, n_estimators=40, random_state=12345)
model.fit(train_features, train_target)
accuracy = model.score(test_features, test_target)
print("Доля правильных ответов на тестовой выборке:", accuracy)
print("Доля правильных ответов на валидной выборке:", best_accuracy_ranfor)

Доля правильных ответов на тестовой выборке: 0.7931570762052877
Доля правильных ответов на валидной выборке: 0.80248833592535


### Проверим модель на логистической регрессии

In [14]:
model = LogisticRegression(solver = 'liblinear')
model.fit(train_features, train_target)
accuracy = model.score(test_features, test_target)
print("Доля правильных ответов на тестовой выборке:", accuracy)
print("Доля правильных ответов на валидной выборке:", accuracy_logreg)

Доля правильных ответов на тестовой выборке: 0.7620528771384136
Доля правильных ответов на валидной выборке: 0.7340590979782271


Произвели проверку моделей на тестовой выборке. Получили, что больше всего значения близки у моделей "случайный лес" и "лигическая регрессия".

## (бонус) Проверьте модели на адекватность

Для проверки моделей на адекватность используем DummyClassifier и его стратегии:
- stratified (генерирует прогнозы с учетом распределения классов обучающей выборки), 
- most_frequent (всегда предсказывает наиболее частую метку в обучающем наборе), 
- uniform (генерирует прогнозы равномерно в случайном порядке).
    Источник: https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html
    
По одной стратении на применяемые нами модели.

In [15]:
fanfor = DummyClassifier(strategy='stratified')
fanfor.fit(train_features, train_target)
accuracy = fanfor.score(test_features, test_target)
print("Доля правильных ответов при проверке на адекватность по стратегии 'stratified':", accuracy)
print("Доля правильных ответов на валидной выборке модели 'дерево решений':", best_accuracy_dec_tree)

Доля правильных ответов при проверке на адекватность по стратегии 'stratified': 0.6049766718506998
Доля правильных ответов на валидной выборке модели 'дерево решений': 0.7480559875583204


In [16]:
fanfor = DummyClassifier(strategy='most_frequent')
fanfor.fit(train_features, train_target)
accuracy = fanfor.score(test_features, test_target)
print("Доля правильных ответов при проверке на адекватность по стратегии 'most_frequent':", accuracy)
print("Доля правильных ответов на валидной выборке модели 'случайный лес':", best_accuracy_ranfor)

Доля правильных ответов при проверке на адекватность по стратегии 'most_frequent': 0.7060653188180405
Доля правильных ответов на валидной выборке модели 'случайный лес': 0.80248833592535


In [17]:
fanfor = DummyClassifier(strategy='uniform')
fanfor.fit(train_features, train_target)
accuracy = fanfor.score(test_features, test_target)
print("Доля правильных ответов при проверке на адекватность по стратегии 'uniform':", accuracy)
print("Доля правильных ответов на валидной выборке модели 'логистическая регрессия':", accuracy_logreg)

Доля правильных ответов при проверке на адекватность по стратегии 'uniform': 0.5178849144634525
Доля правильных ответов на валидной выборке модели 'логистическая регрессия': 0.7340590979782271


Провели проверку применяемых нами моделий на адекватность. Уведели, что у всех моделей доля правильных ответов превосходит долю правильных ответов, полученных на модели выбранной для проверки на адекватность.

Провели проверку на адекватность.

В случае accuracy всегда выгоднее всего выбирать стратегию most_frequent, но проверили разные стратегии.

## Вывод

В ходе нашего исследования были проверены модели дерева решений, случайный лес и логистическая регрессия.
Все модели были проверены на тестовой выборке и прошли проверку на адекватность.

Лучшей показала себя модель случайный лес - доля правильных ответов: 0.7931570762052877, с параметрами:
- глубина дерева: 14,  
- количество деревьев: 45,  
- минимальное число объектов в листье: 4.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [X] Весь код исполняется без ошибок
- [X] Ячейки с кодом расположены в порядке исполнения
- [X] Выполнено задание 1: данные загружены и изучены
- [X] Выполнено задание 2: данные разбиты на три выборки
- [X] Выполнено задание 3: проведено исследование моделей
    - [X] Рассмотрено больше одной модели
    - [X] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [X] Написаны выводы по результатам исследования
- [X] Выполнено задание 3: Проведено тестирование
- [X] Удалось достичь accuracy не меньше 0.75
